In [1]:
from json_processor import build_data_set
from constant import NEW_LOG, OLD_LOG, VERSION_TRAIN, VERSION_TEST
from common import translate_html_entity
import pandas as pd
import os
import logging
import json
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/pkun/.ssh/payuat-985b0c12a3d4.json"
os.chdir('/Users/pkun/PycharmProjects/ui_api_automated_test')
LOG = logging.getLogger('data_build')
chinese_map_english = json.load(open('data/chinese_map_english.json', 'r'))

In [2]:
def translate_seria_text_to_english(seria):
    import six
    from google.cloud import translate_v2 as translate

    translate_client = translate.Client()
    result = dict() 
    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    count = 0
    for data in seria:
        result[data] = translate_client.translate(
            data, target_language='en')['translatedText']
        result[data] = translate_html_entity(result[data])
        count += 1
        if count % 200 == 0:
            print(count)
            print(len(result))
    return result

In [3]:
def translate_single_text_to_english(text):
    return chinese_map_english[text]


In [4]:
# train_indices = np.random.choice(8052, int(8052*0.8), replace=False)
# test_indices = np.delete(np.arange(8052), train_indices)
# train_data = copy_data_set[train_indices]
# test_data = copy_data_set[test_indices] 

In [5]:
def from_data_to_tsv(data_set, file):
    f = open(file, 'w')
    for i in data_set:
        f.write(i.to_text()+'\n')
    f.close()

In [6]:
new_log_data_set = build_data_set(NEW_LOG)
old_log_data_set = build_data_set(OLD_LOG)

In [7]:
new_log_data_set

,label,query,ui_info
0,yes,confirm payment information to confirm payment...,立即支付
1,no,confirm payment information to confirm payment...,"当前所在页面,支付"
2,no,confirm payment information to confirm payment...,返回
3,no,confirm payment information to confirm payment...,支付
4,no,confirm payment information to confirm payment...,body
...,...,...,...
194733,no,user to submit cardholder information when fil...,选择
194734,no,user to submit cardholder information when fil...,同意用户服务协议
194735,no,user to submit cardholder information when fil...,下一步
194736,no,user to submit cardholder information when fil...,将发起0.05美元预授权以验证账户


In [8]:
def remove_duplicates_with_label_eq_no(data_set):
    data_set[data_set['label']=='no'] = data_set[data_set['label']=='no'].drop_duplicates(['query', 'ui_info'])
    data_set.dropna(axis=0, how='any', inplace=True)

In [9]:
remove_duplicates_with_label_eq_no(new_log_data_set)
remove_duplicates_with_label_eq_no(old_log_data_set)
new_log_data_set

,label,query,ui_info
0,yes,confirm payment information to confirm payment...,立即支付
1,no,confirm payment information to confirm payment...,"当前所在页面,支付"
2,no,confirm payment information to confirm payment...,返回
3,no,confirm payment information to confirm payment...,支付
4,no,confirm payment information to confirm payment...,body
...,...,...,...
194526,yes,user to submit cardholder information when fil...,请输入电话
194571,yes,user to submit cardholder information when fil...,请输入电话
194610,yes,user to submit cardholder information when fil...,请输入电子邮件
194655,yes,user to submit cardholder information when fil...,请输入电子邮件


In [10]:
def describe_label_number(data_set):
    print(data_set.head())
    print('Yes', len(data_set[data_set['label'] == 'yes']))
    print('No', len(data_set[data_set['label'] == 'no']))
    data_set

In [11]:
describe_label_number(new_log_data_set)
describe_label_number(old_log_data_set)

  label                                              query    ui_info
0   yes  confirm payment information to confirm payment...       立即支付
1    no  confirm payment information to confirm payment...  当前所在页面,支付
2    no  confirm payment information to confirm payment...         返回
3    no  confirm payment information to confirm payment...         支付
4    no  confirm payment information to confirm payment...       body
Yes 7450
No 7297
  label                                              query    ui_info
0   yes  confirm payment information to confirm payment...       立即支付
1    no  confirm payment information to confirm payment...  当前所在页面,支付
2    no  confirm payment information to confirm payment...         支付
3    no  confirm payment information to confirm payment...         返回
4    no  confirm payment information to confirm payment...       body
Yes 796
No 877


In [12]:
new_log_data_set['ui_info'].head().apply(translate_single_text_to_english)

0      pay immediately
1    Current page, pay
2               return
3               To pay
4                 body
Name: ui_info, dtype: object

In [13]:
new_log_data_set['ui_info'] = new_log_data_set['ui_info'].apply(translate_single_text_to_english)
old_log_data_set['ui_info'] = old_log_data_set['ui_info'].apply(translate_single_text_to_english)

In [21]:
# new_log_data_set.to_csv(path_or_buf=VERSIONT_TRAIN, sep='\t', header=False, index=False)
# old_log_data_set.to_csv(path_or_buf=VERSIONT_TEST, sep='\t', header=False, index=False)
def to_csv(data_set, path):
    data_set.to_csv(path_or_buf=path, sep='\t', header=False, index=False)

In [23]:
train_set = new_log_data_set.copy()
test_set = old_log_data_set.copy()
# remove the data in train
# test_set = pd.concat([test_set, train_set, train_set]).drop_duplicates(keep=False) will remove the data duplicated but not in train
test_set

,label,query,ui_info
0,yes,confirm payment information to confirm payment...,pay immediately
1,no,confirm payment information to confirm payment...,"Current page, pay"
2,no,confirm payment information to confirm payment...,To pay
3,no,confirm payment information to confirm payment...,return
4,no,confirm payment information to confirm payment...,body
...,...,...,...
16340,yes,confirm payment information to confirm payment...,pay immediately
16349,yes,confirm payment information to confirm payment...,pay immediately
16358,yes,confirm payment information to confirm payment...,pay immediately
16367,yes,submit payment password authorized payment vie...,payment method


In [24]:
for index, item in test_set.iterrows():
	item = item.to_frame().T
	if not pd.merge(train_set, item, on=test_set.columns.to_list()).empty:
		test_set.drop(index, inplace=True)

In [26]:
test_set

,label,query,ui_info
9,yes,submit payment password authorized payment vie...,Discount 0.03 yuan
14,no,submit payment password authorized payment vie...,￥0.05
22,no,submit payment password authorized payment vie...,￥0.02
24,no,submit payment password authorized payment vie...,¥0.05
26,no,submit payment password authorized payment vie...,Discount 0.03 yuan
...,...,...,...
15063,yes,submit bank card and identity information subm...,Fill in the address
15532,yes,add a bank card to cancel the submission of ba...,return
15572,yes,select the short phone number to select the mo...,Ping An Bank savings card (0590) reserved mobi...
16092,yes,add a bank card to cancel the submission of ba...,return


In [22]:
to_csv(train_set, 'train.tsv')
to_csv(test_set, 'test.tsv')